In [1]:
import numpy as np
import pandas as pd

## Importing the Dataset

In [4]:
data = pd.read_csv("~/Documents/data/house_data/House_Rent_Dataset.csv")
list(data.columns)

['Posted On',
 'BHK',
 'Rent',
 'Size',
 'Floor',
 'Area Type',
 'Area Locality',
 'City',
 'Furnishing Status',
 'Tenant Preferred',
 'Bathroom',
 'Point of Contact']

In [8]:
column_to_description = []

descriptions_temp = ["Posted On: The date on which the house listing was posted.",
"BHK: Number of Bedrooms, Hall, Kitchen.",
"Rent: Rent of the Houses/Apartments/Flats.",
"Size: Size of the Houses/Apartments/Flats in Square Feet.",
"Floor: Houses/Apartments/Flats situated in which Floor and Total Number of Floors (Example: Ground out of 2, 3 out of 5, etc.)",
"Area Type: Size of the Houses/Apartments/Flats calculated on either Super Area or Carpet Area or Build Area.",
"Area Locality: Locality of the Houses/Apartments/Flats.",
"City: City where the Houses/Apartments/Flats are Located.",
"Furnishing Status: Furnishing Status of the Houses/Apartments/Flats, either it is Furnished or Semi-Furnished or Unfurnished.",
"Tenant Preferred: Type of Tenant Preferred by the Owner or Agent.",
"Bathroom: Number of Bathrooms.",
"Point of Contact: Whom should you contact for more information regarding the Houses/Apartments/Flats."]

for i in range(len(list(data.columns))):
    column_to_description.append([list(data.columns)[i],descriptions_temp[i]])
                                  
print(column_to_description)

[['Posted On', 'Posted On: The date on which the house listing was posted.'], ['BHK', 'BHK: Number of Bedrooms, Hall, Kitchen.'], ['Rent', 'Rent: Rent of the Houses/Apartments/Flats.'], ['Size', 'Size: Size of the Houses/Apartments/Flats in Square Feet.'], ['Floor', 'Floor: Houses/Apartments/Flats situated in which Floor and Total Number of Floors (Example: Ground out of 2, 3 out of 5, etc.)'], ['Area Type', 'Area Type: Size of the Houses/Apartments/Flats calculated on either Super Area or Carpet Area or Build Area.'], ['Area Locality', 'Area Locality: Locality of the Houses/Apartments/Flats.'], ['City', 'City: City where the Houses/Apartments/Flats are Located.'], ['Furnishing Status', 'Furnishing Status: Furnishing Status of the Houses/Apartments/Flats, either it is Furnished or Semi-Furnished or Unfurnished.'], ['Tenant Preferred', 'Tenant Preferred: Type of Tenant Preferred by the Owner or Agent.'], ['Bathroom', 'Bathroom: Number of Bathrooms.'], ['Point of Contact', 'Point of Cont